In [ ]:
%pip install flask flask-cors numpy pandas scikit-learn nltk matplotlib requests lightgbm joblib

from flask import Flask, jsonify, request, send_file  # Importa las clases y funciones necesarias de Flask
from flask_cors import CORS  # Importa CORS para permitir solicitudes desde otros dominios
import pandas as pd  # Importa pandas para el manejo de datos
import joblib  # Importa joblib para cargar modelos entrenados

# Importa funciones y modelos de los módulos correspondientes
from terremotos import modelo_dano, preproceso_dano, predecir_dano  # Para cargar el modelo de terremotos
from frontend import frontend_bp  # Importa el blueprint desde frontend.py

# Inicializa la aplicación Flask
app = Flask(__name__)
CORS(app)  # Habilita CORS para la aplicación

# Registra el blueprint del frontend
app.register_blueprint(frontend_bp)


# --- Función de predicción (Endpoint de la ruta) ---

@app.route('/api/terremotos', methods=['POST'])
def predict_damage():
    """
    Ruta para predecir el grado de daño (1, 2, o 3) basado en las características del edificio.
    
    Espera un cuerpo JSON con una lista de diccionarios, donde cada diccionario 
    contiene las 38 características de un edificio.
    Ejemplo de datos recibidos: [{"geo_level_1_id": 8, "count_floors_pre_eq": 2, ...}, ...]
    """
    if modelo_dano is None or preproceso_dano is None:
        return jsonify({"error": "Modelo o Procesador no cargado. Revisa la consola del servidor."}), 500

    try:
        # Los datos se reciben como una lista de objetos JSON (lo que envía el front-end)
        datos = request.json
        
        if not datos or not isinstance(datos, list):
             return jsonify({"error": "El cuerpo de la solicitud debe ser una lista de características JSON."}), 400

        # Convertir la lista de objetos a un DataFrame de Pandas
        df_nuevo = pd.DataFrame(datos)
        
        prediccion = predecir_dano(modelo_dano, preproceso_dano, df_nuevo)
        
        # Devolver el resultado como JSON
        return jsonify({"predictions": prediccion.tolist()})

    except Exception as e:
        print(f"Error durante la predicción: {e}")
        return jsonify({"error": f"Error interno del servidor durante la predicción: {e}"}), 500

# Inicia la aplicación Flask en el puerto 5000
if __name__ == "__main__":
    app.run(port=5000)



[notice] A new release of pip available: 22.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Modelo y procesador cargados desde archivos existentes.
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Dec/2025 14:40:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2025 14:40:19] "GET /favicon.ico/ HTTP/1.1" 404 -
127.0.0.1 - - [16/Dec/2025 14:40:24] "GET /terremotos/ HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2025 14:40:24] "GET /terremotos/index.css HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2025 14:40:24] "GET /variables.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2025 14:40:24] "GET /terremotos/index.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Dec/2025 14:40:35] "POST /api/terremotos HTTP/1.1" 200 -
